In [1]:
import pandas as pd
import gurobipy as gp
import numpy as np
from gurobipy import *
from geopy.distance import geodesic
import math

In [2]:
# read csv files
#zipcode = pd.read_csv('allegheny_zipcodes.csv')
#pods = pd.read_csv('processed_data/Allegheny_County_POD_Sites.csv')

# dataframe for zipcode wise population
population = pd.read_csv('allegheny_population.csv')

# dataframe for distance between zipcode centroids and POD sites
distance = pd.read_csv('distance.csv').iloc[:,1:]
distance.columns = [i for i in range(distance.shape[1])]

# dataframe for list of zip codes with outbreak
outbreak = pd.read_csv('outbreak.csv')

In [3]:
num_areas = range(distance.shape[0])
num_pods = range(distance.shape[1])

In [4]:
# Requirements for vaccination campaign
sumPopulation = sum(population['population'])
maxCapacity = sumPopulation * 0.05
h1, h2, h3, h4, h5, h6, h7, h8 = 1, 500, 700, 900, 10, 50000, 70000, 90000
T1 = 1
T2 = 0
T3 = 1
max_budget = 50*10**6

"""
some good ranges for this dataset:
max_budget: 50 million to 15 million
maxCapacity: 0.05 to 0.025
setObjective: travel_dist or total_cost or a linear combination of both
"""

'\nsome good ranges for this dataset:\nmax_budget: 50 million to 15 million\nmaxCapacity: 0.05 to 0.025\nsetObjective: travel_dist or total_cost or a linear combination of both\n'

## model 1: baseline - minimizing total travel distance


In [5]:
percent_values = ['min100' , 'min60', 'min30']

builtsite_1_0 = np.zeros((3, distance.shape[1]))

for percent_val in percent_values:
    
    minPercent = outbreak[percent_val]

    # set up the model
    m = Model("facility")

    built = m.addVars(num_pods, vtype=GRB.BINARY) #1 if built; X_j, 47
    assign = m.addVars(num_pods, num_areas, lb = 0.0) #F_ij, 47 * 98
    capacity = m.addVars(num_pods, lb = 0.0, vtype=GRB.INTEGER)
    total_cost = m.addVar(name = "total_cost")


    travel_dist = gp.quicksum(assign[pod,area] * 
                              distance[pod][area] * 
                              int(population.iloc[area]['population']) for area in num_areas for pod in num_pods)

    m.setObjective(0.5 * travel_dist + 0.5 * total_cost, GRB.MINIMIZE)


    # add constraints

    # number of pods constraint
    # m.addConstr(sum(built[j] for j in num_pods) == 20)

    # no assignment, travel should appear without a POD built
    for area in num_areas:
        for pod in num_pods:
            m.addConstr(assign[pod, area] <= built[pod])

    # capacity of a pod should be zero for the locations where the POD is not built
    bigm = sumPopulation

    for pod in num_pods:
        m.addConstr(capacity[pod] <= built[pod] * bigm)


    # upper bound for assigning fraction of population to a POD:
    for area in num_areas:
        for pod in num_pods:
            m.addConstr(assign[pod, area] <= 1)


    # Capacity limit for all PODs: capacity of all PODs should be less than maxCapacity
    for pod in num_pods:
        m.addConstr(capacity[pod] <= maxCapacity)


    # Capacity constraint: population assigned to a pod should be smaller than the POD capacity
    m.addConstrs(gp.quicksum(int(population.iloc[area]['population']) * assign[pod, area] for area in num_areas)
                     <= capacity[pod] for pod in num_pods)


    # minimum coverage constraint
    needs = sum(minPercent[i]* population['population'][i] for i in num_areas)
    #print(needs)
    m.addConstr(sum(capacity[pod] for pod in num_pods) >= needs )

    # population coverage constraint
    for area in num_areas:
        m.addConstr(gp.quicksum(assign[pod, area] for pod in num_pods) >= 1 * minPercent[area])

    # cost related constraints:

    m.addConstr(gp.quicksum((h1 * capacity[pod] + h2 * T1 + h3 * T2 + h4 * T3) * built[pod] +
                             (h5 * capacity[pod] + h6 * T1 + h7 * T2 + h8 * T3) * built[pod] for pod in num_pods) == total_cost)

    m.addConstr(total_cost <= max_budget)


    m.Params.TimeLimit = 60
    # Solve
    m.optimize()

    assign_total = {} # dict of pod: assigned population
    for pod in num_pods:
        total_pop = 0
        for area in num_areas:
            total_pop += population.iloc[area]['population'] * assign[pod,area].X
        assign_total[pod] = total_pop

    #sum(assign_total.values())
    #sumPopulation

    # cost results:
    cost_dict = {}
    for pod in num_pods:
        a_pod = (h1 * capacity[pod].X + h2 * T1 + h3 * T2 + h4 * T3) * built[pod].X
        b_pod = (h5 * capacity[pod].X + h6 * T1 + h7 * T2 + h8 * T3) * built[pod].X
        cost_dict[pod] = a_pod + b_pod

    #sum(cost_dict.values())


    # total cost
    #print(f"Total cost: {total_cost.X}")

    #travel distance:
    total_dist_opt = 0 # dict of pod: assigned population
    for pod in num_pods:
        for area in num_areas:
            total_dist_opt += (assign[pod,area].X * 
                               distance[pod][area] * 
                               int(population.iloc[area]['population']))

    #total distance travelled
    #print(f"Total distance travelled: {total_dist_opt}")

    builtsite=[]
    for i in num_pods:
        if built[i].X >0.01:
            print(i)
            builtsite.append(i)
    builtsite
    # 100 [4, 6, 7, 10, 18, 19, 20, 25, 27, 29, 30, 31, 34, 35, 42, 43, 47]
    # 60 [3, 6, 10, 16, 17, 26, 30, 36, 39, 40]
    # 30 [8, 9, 18, 24, 31]

    for i in builtsite:
        builtsite_1_0[percent_values.index(percent_val)][i] = 1


pd.DataFrame(builtsite_1_0.T).to_csv('model_1_POD_plan_output.csv')



Academic license - for non-commercial use only - expires 2022-09-10
Using license file C:\Users\Ravi\gurobi.lic
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9453 rows, 4701 columns and 23266 nonzeros
Model fingerprint: 0xf74a9653
Model has 1 quadratic constraint
Variable types: 4607 continuous, 94 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [5e-01, 5e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.02s
Presolved: 4941 rows, 4748 columns, 19036 nonzeros
Variable types: 4606 continuous, 142 integer (47 binary)

Root relaxation: objective 2.244824e+06, 1273 iterations, 0.04 seconds

    Node

H    0     0                    2392034.5838 814000.444  66.0%     -    0s
     0     0 886436.793    0   21 2392034.58 886436.793  62.9%     -    0s
     0     0 956486.047    0   21 2392034.58 956486.047  60.0%     -    0s
H    0     0                    2364657.7757 956486.047  59.6%     -    0s
     0     0 1066868.48    0   17 2364657.78 1066868.48  54.9%     -    0s
     0     0 1168449.57    0   21 2364657.78 1168449.57  50.6%     -    0s
H    0     0                    2364377.9123 1168449.57  50.6%     -    0s
H    0     0                    2310471.5113 1168449.57  49.4%     -    0s
     0     0 1459761.96    0   28 2310471.51 1459761.96  36.8%     -    0s
     0     0 1684695.49    0   33 2310471.51 1684695.49  27.1%     -    0s
H    0     0                    2266208.8669 1684695.49  25.7%     -    0s
     0     0 1858864.30    0   27 2266208.87 1858864.30  18.0%     -    0s
H    0     0                    2184236.7576 1858864.30  14.9%     -    0s
     0     0 2050082.44  

In [6]:
"""
#population assignment visualization
for j in num_areas:
    for i in num_pods:
        if assign[i, j].x > 0:
            print("area:" + str(j) + '\t->\t' + "POD:" + str(i) + ' ' + str(assign[i, j].x))
"""

'\n#population assignment visualization\nfor j in num_areas:\n    for i in num_pods:\n        if assign[i, j].x > 0:\n            print("area:" + str(j) + \'\t->\t\' + "POD:" + str(i) + \' \' + str(assign[i, j].x))\n'

## model 2: baseline - minimizing maximum travel distance by an individual


In [7]:
builtsite_1_0 = np.zeros((3, distance.shape[1]))

for percent_val in percent_values:
    
    minPercent = outbreak[percent_val]

    # set up the model
    m2 = Model("facility")

    built = m2.addVars(num_pods, vtype=GRB.BINARY) #1 if built; X_j, 47
    assign = m2.addVars(num_pods, num_areas, lb = 0.0) #F_ij, 47 * 98
    assign_1_0 = m2.addVars(num_pods, num_areas, lb = 0.0, vtype=GRB.INTEGER)
    capacity = m2.addVars(num_pods, lb = 0.0, vtype=GRB.INTEGER)
    max_travel_dist = m2.addVar(name = "max_travel_dist", lb = 0.0)
    total_cost = m2.addVar(name = "total_cost", lb = 0.0)

    # set up objective

    #max_travel_dist = max(distance[pod][area] * built[pod] for area in num_areas for pod in num_pods)

    m2.setObjective(0.5 * max_travel_dist + 0.5 * total_cost, GRB.MINIMIZE)


    # add constraints

    # number of pods constraint
    # m.addConstr(sum(built[j] for j in num_pods) == 20)

    # no assignment, travel should appear without a POD built
    for area in num_areas:
        for pod in num_pods:
            m2.addConstr(assign[pod, area] <= built[pod])

    # capacity of a pod should be zero for the locations where the POD is not built
    bigm = sumPopulation

    for pod in num_pods:
        m2.addConstr(capacity[pod] <= built[pod] * bigm)


    # upper bound for assigning fraction of population to a POD:
    for area in num_areas:
        for pod in num_pods:
            m2.addConstr(assign[pod, area] <= 1)


    # Capacity limit for all PODs: capacity of all PODs should be less than maxCapacity
    for pod in num_pods:
        m2.addConstr(capacity[pod] <= maxCapacity)


    # Capacity constraint: population assigned to a pod should be smaller than the POD capacity
    m2.addConstrs(gp.quicksum(int(population.iloc[area]['population']) * assign[pod, area] for area in num_areas)
                     <= capacity[pod] for pod in num_pods)

    # minimum coverage constraint
    needs = sum(minPercent[i]* population['population'][i] for i in num_areas)
    print(needs)
    m2.addConstr(sum(capacity[pod] for pod in num_pods) >= needs )


    # population coverage constraint
    for area in num_areas:
        m2.addConstr(gp.quicksum(assign[pod, area] for pod in num_pods) >= 1 * minPercent[area])

    """ to apply the above constraint on overall population instead of separately on each zipcode -
    m2.addConstr(gp.quicksum(gp.quicksum(assign[pod, area] 
                                         for pod in num_pods) * 
                             int(population.iloc[area]['population']) 
                             for area in num_areas) >= minPercent * sumPopulation)
    """


    # one person gets assigned at max only once
    for area in num_areas:
        m2.addConstr(gp.quicksum(assign[pod, area] for pod in num_pods) <= 1)


    # assign_1_0
    for area in num_areas:
        for pod in num_pods:
            m2.addConstr(assign_1_0[pod, area] >= assign[pod, area])

    # maximum travel distance for an individual 
    for area in num_areas:
        for pod in num_pods:
            m2.addConstr(assign_1_0[pod,area] * distance[pod][area] <= max_travel_dist)

    m2.addConstr(gp.quicksum((h1 * capacity[pod] + h2 * T1 + h3 * T2 + h4 * T3) * built[pod] +
                             (h5 * capacity[pod] + h6 * T1 + h7 * T2 + h8 * T3) * built[pod] for pod in num_pods) == total_cost)

    m2.addConstr(total_cost <= max_budget)


    m2.Params.TimeLimit = 60
    # Solve
    m2.optimize()


    assign_total = {} # dict of pod: assigned population
    for pod in num_pods:
        total_pop = 0
        for area in num_areas:
            total_pop += population.iloc[area]['population'] * assign[pod,area].X
        assign_total[pod] = total_pop


    #sum(assign_total.values())

    #sum(assign_total.values())/sumPopulation

    #max_travel_dist.X


    builtsite=[]
    for i in num_pods:
        if built[i].X >0.01:
            print(i)
            builtsite.append(i)
    #builtsite
    # 100 [4, 6, 7, 10, 18, 19, 20, 25, 27, 29, 30, 31, 34, 35, 42, 43, 47]
    # 60 [3, 6, 10, 16, 17, 26, 30, 36, 39, 40]
    # 30 [8, 9, 18, 24, 31]

    for i in builtsite:
        builtsite_1_0[percent_values.index(percent_val)][i] = 1


pd.DataFrame(builtsite_1_0.T).to_csv('model_2_POD_plan_output.csv')





963380.0000000001
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 18763 rows, 9308 columns and 46296 nonzeros
Model fingerprint: 0x8c93570b
Model has 1 quadratic constraint
Variable types: 4608 continuous, 4700 integer (47 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+06]
  QMatrix range    [1e+01, 1e+01]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [5e-01, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+07]
Presolve removed 4654 rows and 0 columns
Presolve time: 0.10s
Presolved: 14251 rows, 9355 columns, 46614 nonzeros
Variable types: 4607 continuous, 4748 integer (4653 binary)

Root relaxation: objective 1.131205e+06, 17145 iterations, 0.47 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  

In [8]:
"""
for j in num_areas:
    for i in num_pods:
        if assign[i, j].x > 0:
            print("area:" + str(j) + '->' + "POD:" + str(i) + ' ' + str(assign[i, j].x))
            
"""

'\nfor j in num_areas:\n    for i in num_pods:\n        if assign[i, j].x > 0:\n            print("area:" + str(j) + \'->\' + "POD:" + str(i) + \' \' + str(assign[i, j].x))\n            \n'